In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt

%matplotlib inline
plt.rcParams['figure.figsize'] = (17,9)
plt.rcParams['font.size'] = 11
pd.set_option("display.max_rows", None, "display.max_columns", None)
plt.rcParams['figure.dpi'] = 70
sns.set_style('darkgrid')
import warnings
warnings.filterwarnings("ignore")


# Problem statement
Real estate transactions are quite opaque sometimes and it
may be difficult for a newbie to know the fair price of any given
home. Thus, multiple real estate websites have the functionality
to predict the prices of houses given different features
regarding it. Such forecasting models will help buyers to identify
a fair price for the home and also give insights to sellers as to
how to build homes that fetch them more money. Chennai
house sale price data is shared here and the participants are
expected to build a sale price prediction model that will aid the
customers to find a fair price for their homes and also help the
sellers understand what factors are fetching more money
for the houses?


understanding Attributes in the data:
PART_Id: Id of the particular house i.e. Project ID

AREA: The area where house is situated

INT_SQFT: Total area of the house in square feet.

DATE_SALE: The date in which the house got sold

DIST_MAINROAD: How many meters the house is far from main road

N_BEDROOM: Total Number of Bedrooms

N_BATHROOM: Total Number of bathrooms

SALE_COND: Types of sale

PARK_FACIL: Parking facility

DATE_BUILD: The date in which the house was built

BUILDTYPE: What type of building it is i.e. residential or commercial

UTILITY_AVAIL: What are the public facilities are available

STREET: Type of street(road) the house is situated

MZZONE: What zone the house belongs to. (There are currently 13 zones in Chennai)

QS_ROOMS: Masked rooms

QS_BATHROOM: Masked bathrooms

QS_BEDROOM: Masked bedroomU

QS_OVERALL:

REG_FEE: Total registration fee offered to register the house

COMMIS: Total commission payed

SALES_PRICE: Sale price of the house


### Loading Dataset

In [ ]:
data = pd.read_csv("chennai.csv")
data.head()

### Getting information about data

In [ ]:
data.columns

In [ ]:
data.shape

In [ ]:
data.info()


In [ ]:
data.describe()


## Data cleaning

##### Checking and filling null values

In [ ]:
#checking null values
data.isnull().sum()

In [ ]:
data.isnull().sum()/data.shape[0]*100


##### so we can see there are missing values in only three columns : N_beroom- 0.014% , N_bathroom:-0.07% and Qs overall :- 0.67%

In [ ]:
sns.heatmap(data.isnull())
plt.show()
#visualising missing values

#### filling missing values

In [ ]:
print(data.N_BATHROOM.isnull().sum())

In [ ]:
print("mode:",data.N_BATHROOM.mode())
print("mean:",data.N_BATHROOM.mean())

In [ ]:
data.N_BATHROOM.fillna(1,inplace = True)

we are filling the missing values in N_bthroom columns as 1 because the mean is 1.21 and no of bathroom cant be 1.21 so we will fill with median

In [ ]:
data.N_BATHROOM.isnull().sum()

In [ ]:
data.N_BEDROOM.isnull().sum()

In [ ]:
data.N_BEDROOM.fillna(1, inplace = True)

In [ ]:
data.QS_OVERALL.isnull().sum()

In [ ]:
data.QS_OVERALL.mean()

In [ ]:
data.QS_OVERALL.fillna(3.50, inplace = True)

In [ ]:
sns.heatmap(data.isnull(),cmap = "YlOrRd")
plt.show()


##### as we can see our data frame is free from null values

### Finding spelling mistakes

#### find catagorical and numerical columns

In [ ]:
cat_col = data.select_dtypes("object")

In [ ]:
num_col = data.select_dtypes(exclude=['object'])

In [ ]:
num_col.head()

In [ ]:
cat_col.head()

In [ ]:
cat_col.shape

In [ ]:
cat_col.AREA.value_counts()

#### correcting speliings of catogoricals values

In [ ]:
new_area = {
    "Chrompt":"Chrompet",
    "Chrmpet":"Chrompet",
    "Chrmpet" :"Chrompet",
    "Chormpet":"Chrompet",
    "TNagar":"T Nagar",
    "Ana Nagar":"Anna Nagar",
    "Ann Nagar":"Anna Nagar",
    "Karapakam":"Karapakkam",
    "Velchery":"Velachery",
    "KKNagar":"KK Nagar",
    "Adyr":"Adyar",
    "Adayar":"Adyar"

}

In [ ]:
data['AREA'] = data.AREA.replace(new_area)

In [ ]:
data.AREA.value_counts()


In [ ]:
cat_col.SALE_COND.value_counts()

In [ ]:
sl_cd = {
    "Adj Land":"AdjLand",
    "Ab Normal":"AbNormal",
    "Partiall":"Paratial",
    "PartiaLl":"Partial",
    "Paratial":"Partial",

}

In [ ]:
data["SALE_COND"] = data.SALE_COND.replace(sl_cd)

In [ ]:
data.SALE_COND.value_counts()

In [ ]:
data.PARK_FACIL.value_counts()

In [ ]:
data["PARK_FACIL"] = data.PARK_FACIL.replace({"Noo":'No'})

In [ ]:
data.PARK_FACIL.value_counts()

In [ ]:
data.BUILDTYPE.value_counts()

In [ ]:
data["BUILDTYPE"]=data.BUILDTYPE.replace({
    "Other":"Others",
    "Comercial":"Commercial"
})

In [ ]:
data.BUILDTYPE.value_counts()

In [ ]:
data.UTILITY_AVAIL.value_counts()

In [ ]:
data["UTILITY_AVAIL"] = data.UTILITY_AVAIL.replace({"All Pub":"AllPub"})

In [ ]:
data.UTILITY_AVAIL.value_counts()

In [ ]:
data.STREET.value_counts()

In [ ]:
data["STREET"] = data.STREET.replace({"Pavd":"Paved",
                                       "NO Access":"No Access","NoAccess":"No Access"})

In [ ]:
data.STREET.value_counts()

Here we corrected spellings of all catagorical values

In [ ]:
#2.convert date time coumns to datetime type
data['DATE_SALE']=pd.to_datetime(data.DATE_SALE)
data['DATE_BUILD']=pd.to_datetime(data.DATE_BUILD)

In [ ]:
data['BUILD_YEAR'] = data['DATE_BUILD'].dt.year
data['SALE_YEAR'] =data['DATE_SALE'].dt.year

#### Checking for duplicate values

In [ ]:
data.duplicated().any()

#### Removing unwanted column

In [ ]:
df=data.drop(['DATE_SALE',"DATE_BUILD"],axis = 1)

In [ ]:
df = data.drop(['PRT_ID','REG_FEE','COMMIS'],axis=1)


In [ ]:
df.drop(["DATE_SALE"],axis =1)

In [ ]:
df.columns

In [ ]:
df.shape

Here no duplicate values found

## Visualising Data

In [ ]:
df.keys()

1.AREA

AS we can see here no.of houses are more in chrompet and karapakkam as price is low here as compared to anna nagar and t nagar where price of houses are quite high and no. of houses are low .

In [ ]:
sns.countplot(data=df,x='AREA',palette='viridis');
plt.title('NO OF HOUSES IN EACH AREA')
plt.show()

In [ ]:
sns.barplot(data=df,x = 'AREA',y="SALES_PRICE",palette='Accent');
plt.title('AREA v/s SALES PRICE')

In [ ]:
plt.figure(figsize=(18,10))
sns.barplot(data=df,x = 'AREA',y="SALES_PRICE",hue='N_BEDROOM',palette='prism')

In [ ]:
sns.barplot(data=df,x = 'AREA',y="SALES_PRICE",hue='N_ROOM',palette='prism')

In [ ]:
sns.barplot(data=df,x = 'AREA',y="SALES_PRICE",hue='PARK_FACIL',palette='spring')

In [ ]:
sns.barplot(data=df,x = 'AREA',y="SALES_PRICE",hue='BUILDTYPE',palette='prism')

In [ ]:
sns.barplot(data=df,x = 'AREA',y='INT_SQFT',palette='rainbow')

In [ ]:
sns.barplot(data=df,x = 'AREA',y="SALES_PRICE",hue='STREET',palette='hsv')

In [ ]:
df.keys()

In [ ]:
sns.lineplot(data= df,x= 'AREA',y='QS_OVERALL')

From above analysis we understand following points

1.Chrompet and karapakkam have more no. of houses as compared to other areas.

2.Price of houses are higher in T nagar followed by anna nagar.

3.Floor area in sqft. is more in KK nagar. Velachery,T nagar and anna nagr have houses of same areas which is less than kk nagar.

4 . Commercial type of building getting more price than houses in all localities. Anna nagar and T nagar has highest price for Commercial type of buildings.


In [ ]:
sns.set_theme(style="darkgrid",palette="hsv")
plt.figure(figsize=(20,16))
plt.subplot(231)
sns.barplot(x='AREA',y='SALES_PRICE',data=df,order=df.groupby('AREA')['SALES_PRICE'].mean().reset_index().sort_values('SALES_PRICE')['AREA'])
plt.xticks(rotation=15)
plt.title('Sales price v/s area')
plt.subplot(232)
sns.barplot(x='BUILDTYPE',y='SALES_PRICE',data=df,order=df.groupby('BUILDTYPE')['SALES_PRICE'].mean().reset_index().sort_values('SALES_PRICE')['BUILDTYPE'])
plt.xticks(rotation=15)
plt.title('Buildtype v/s sales price')
plt.subplot(233)
sns.barplot(x='UTILITY_AVAIL',y='SALES_PRICE',data=df,order=df.groupby('UTILITY_AVAIL')['SALES_PRICE'].mean().reset_index().sort_values('SALES_PRICE')['UTILITY_AVAIL'])
plt.xticks(rotation=15)
plt.title('utility available v/s sales price')
plt.subplot(234)
sns.barplot(x='STREET',y='SALES_PRICE',data=df,order=df.groupby('STREET')['SALES_PRICE'].mean().reset_index().sort_values('SALES_PRICE')['STREET'])
plt.xticks(rotation=15)
plt.title('Type of stree available v/s sales price')
plt.subplot(235)
sns.barplot(x='PARK_FACIL',y='SALES_PRICE',data=df)
plt.xticks(rotation=15)
plt.title('Parking availble v/s Sales Price')
plt.subplot(236)
sns.barplot(x='MZZONE',y='SALES_PRICE',data=df)
plt.xticks(rotation=15)
plt.title('Municiple zone vs sales price')
plt.show()

In [ ]:
sns.barplot(x='N_BEDROOM',y='SALES_PRICE',data=df,palette = 'RdGy',order=df.groupby('N_BEDROOM')['SALES_PRICE'].mean().reset_index().sort_values('SALES_PRICE')['N_BEDROOM'])

plt.show()

In [ ]:
sns.countplot(data=data, x='SALE_YEAR', palette='gist_rainbow')
plt.title("Number of Houses sold per year")
plt.show()

In [ ]:
sns.set_palette('Accent')
ax = sns.regplot(data=data, x='BUILD_YEAR', y='SALES_PRICE')
plt.title("Built year of house Vs. sales price")
plt.show()

In [ ]:
sns.lineplot(data=data, x='BUILD_YEAR', y='SALES_PRICE', color='blue')
plt.title("Yearwise house sales price")
plt.show()

In [ ]:
plt.figure(figsize=(20,12))
sns.barplot(data=data, x='BUILD_YEAR', y='SALES_PRICE',hue = 'AREA')
plt.title("Yearwise house sales price")
plt.xticks(rotation=70)
plt.show()

In [ ]:
sns.lineplot(data=data, x='SALE_YEAR', y='SALES_PRICE', color='green')
plt.title("Yearwise house sales price")
plt.show()

In [ ]:
sns.barplot(x='N_BATHROOM',y='SALES_PRICE',data=df,palette = 'rocket',order=df.groupby('N_BATHROOM')['SALES_PRICE'].mean().reset_index().sort_values('SALES_PRICE')['N_BATHROOM'])



In [ ]:
sns.heatmap(df.corr(),annot= True);

In [ ]:
plt.figure(figsize=(20,20))
sns.pairplot(df,x_vars=['INT_SQFT', 'DIST_MAINROAD', 'N_BEDROOM', 'N_BATHROOM', 'N_ROOM','QS_ROOMS', 'QS_BATHROOM', 'QS_BEDROOM', 'QS_OVERALL'],
             y_vars=["SALES_PRICE"],palette='cool',kind="reg")
plt.show()

In [ ]:
plt.figure(figsize=(17,8))
num_col.plot.box(grid = True);
plt.xticks(rotation =80);
plt.show()

In [ ]:
sns.boxplot(df.SALES_PRICE)

In [ ]:
sns.distplot(df.SALES_PRICE)

##### only sales price has outliers but we cant remove these outliers as it it most important column for us

1. T nagar has highest avg ,selling price and karapakkam has lowest avg selling price. so concentration of house is more in area having low price homes.

2. commercial building prices are higher than residential buildings.

3. utilities does not affect too much on sales price.

4. houses having paved and gravel road around it has little bit costlier.

5. building with parking facility does not affect on sales price too much.

6. RM,RL,RH zones having higher house cost than other zones.

7. More the no. rooms more the price.

8. higher floor area,higher the sales price

### DATA PREPROCESSING

In [ ]:
df.keys()

In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
le = LabelEncoder()

In [ ]:
cat_col.keys()

In [ ]:
df['PARK_FACIL']=le.fit_transform(df['PARK_FACIL'])
df['AREA']=le.fit_transform(df['AREA'])
df['SALE_COND']=le.fit_transform(df['SALE_COND'])
df['UTILITY_AVAIL']=le.fit_transform(df['UTILITY_AVAIL'])
df['STREET']=le.fit_transform(df['STREET'])
df['MZZONE']=le.fit_transform(df['MZZONE'])

In [ ]:
df.shape

In [ ]:
df.head()

In [ ]:
df.STREET.head()

#### One Hot Encoding

In [ ]:
df = pd.get_dummies(df)

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df.keys()


In [ ]:
#dropping unwanted columns
df.drop(['DATE_SALE','DIST_MAINROAD','DATE_BUILD','QS_ROOMS', 'QS_BATHROOM','QS_BEDROOM', 'QS_OVERALL'],axis=1,inplace =True)

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
df.to_csv('chennai_final.csv', index=False)

In [ ]:
## importing final dataset
df = pd.read_csv("chennai_final.csv")

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
plt.figure()
plt.subplot(231)
plt.xlabel("INT_SQFT")
df.INT_SQFT.plot(kind='kde')
plt.subplot(232)
df.SALES_PRICE.plot(kind='kde')
plt.xlabel('SALES_PRICE')
plt.show()


In [ ]:
df['INT_SQFT'] = np.log(df.INT_SQFT)

In [ ]:
df.INT_SQFT.plot(kind = 'kde')
plt.title('INT_SQFT distribution after log tranceformation')

After log tranceformation the distribution looks better than before.

# Machine learning part

In [ ]:
#from sklearn.model_selection import train_test_split
#from sklearn.linear_model import LinearRegression
#from sklearn.neighbors import KNeighborsRegressor
#from sklearn.tree import DecisionTreeRegressor
#from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor
#from sklearn.preprocessing import StandardScaler
#from sklearn.model_selection import GridSearchCV, cross_val_score, RepeatedKFold
#from sklearn.model_selection import RepeatedStratifiedKFold
#from sklearn import metrics

### Tain test split

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X = df.drop(['SALES_PRICE'],axis = 1)

In [ ]:
y = df.SALES_PRICE

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(X,y,shuffle = True, random_state =45,test_size=0.2)

### Feature Scaling
 we use this method to normalize the range of independent variables or features of data.

In [ ]:
from sklearn.preprocessing import StandardScaler


In [ ]:
scl = StandardScaler()

In [ ]:
X_train =scl.fit_transform(X_train)
X_test = scl.transform(X_test)

## Model 1 Simple linear regression

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
lr = LinearRegression().fit(X_train,y_train)


In [ ]:
y_pred =lr.predict(X_test)

Evaluation metrics for linear regression

In [ ]:
from sklearn import metrics

In [ ]:
lr_score = metrics.r2_score(y_test,y_pred)
lr_mae = metrics.mean_absolute_error(y_test,y_pred)

In [ ]:
print("Score on train data: {:.2f}".format(lr.score(X_train, y_train)))
print("Score on Train data: {:.2f}".format(lr.score(X_test, y_test)))
print("R2 score: {:.2f}".format(metrics.r2_score(y_test, y_pred)))
print("MSE: {:.2f}".format(metrics.mean_squared_error(y_test, y_pred)))
print("MAE: {:.2f}".format(metrics.mean_absolute_error(y_test, y_pred)))

In [ ]:
lr_prediction = pd.DataFrame({
    'actual':y_test,
    'predicted':y_pred.flatten()
})

lr_prediction.head()

In [ ]:
plt.figure(figsize=(15,4))
sns.kdeplot(data=lr_prediction, x='actual', label='actual', shade=True)
sns.kdeplot(data=lr_prediction, x='predicted', label='predicted', shade=True)
plt.title("Actual Price Vs Predicted Price")
plt.legend()
plt.show()

In [ ]:
# coefficients
lr_coefficient = pd.Series(lr.coef_, index=X.columns.tolist())

plt.figure(figsize=(15, 4))
lr_coefficient.plot(kind='bar', color=['red'])
plt.title("Coefficients of attributes for linear regressor")
plt.show()


### Random Forest Regressor

In [ ]:
from sklearn.ensemble import RandomForestRegressor

In [ ]:
rf_regressor = RandomForestRegressor(random_state=0).fit(X_train, y_train)

rf_y_predict = rf_regressor.predict(X_test)


# score
print("RF Training score: {:.2f}".format(rf_regressor.score(X_train, y_train)))
print("RF Test score: {:.2f}".format(rf_regressor.score(X_test, y_test)))


print("")
print("RF R2 score: {:.2f}".format(metrics.r2_score(y_test, rf_y_predict)))
print("RF MSE: {:.2f}".format(metrics.mean_squared_error(y_test, rf_y_predict)))
print("RF MAE: {:.2f}".format(metrics.mean_absolute_error(y_test, rf_y_predict)))

In [ ]:


rf_prediction = pd.DataFrame({
    'actual':y_test,
    'predicted': rf_y_predict.flatten()
})


plt.figure(figsize=(15,4))
sns.kdeplot(data=rf_prediction, x='actual', label='actual', shade=True)
sns.kdeplot(data=rf_prediction, x='predicted', label='predicted', color='#e74c3c', shade=True)
plt.title("Actual Price Vs Predicted Price by RF Regressor")
plt.legend()
plt.show()

#### Cross validation of random forest regressor

In [ ]:
rf_search = GridSearchCV(
    estimator=RandomForestRegressor(),
    param_grid={
        'max_depth': range(5,10),
        'n_estimators': (90,100),
        'random_state': [0,20,30]
    },
    cv=5,
    scoring='neg_mean_squared_error',
    verbose=0,
    n_jobs=-1
)

grid_result = rf_search.fit(X,y)

In [ ]:

print("Best estimator for RF: {}".format(grid_result.best_estimator_))


In [ ]:
# initiating the model with best estimators
rf_regressor = RandomForestRegressor(max_depth=9, n_estimators=100, random_state=0)
# fit the model with training data
rf_regressor.fit(X_train, y_train)
# predict on X_test
rf_y_predict = rf_regressor.predict(X_test)

# score
rf_score = metrics.r2_score(y_test, rf_y_predict)
rf_mae = metrics.mean_absolute_error(y_test, rf_y_predict)

print("RF Training score: {:.2f}".format(rf_regressor.score(X_train, y_train)))
print("RF Test score: {:.2f}".format(rf_regressor.score(X_test, y_test)))


print("")
print("RF R2 score: {:.2f}".format(metrics.r2_score(y_test, rf_y_predict)))
print("RF MSE: {:.2f}".format(metrics.mean_squared_error(y_test, rf_y_predict)))
print("RF MAE: {:.2f}".format(metrics.mean_absolute_error(y_test, rf_y_predict)))

In [ ]:
rf_prediction = pd.DataFrame({
    'actual':y_test,
    'predicted': rf_y_predict.flatten()
})


plt.figure(figsize=(15,4))
sns.kdeplot(data=rf_prediction, x='actual', label='actual', shade=True)
sns.kdeplot(data=rf_prediction, x='predicted', label='predicted', color='#e74c3c', shade=True)
plt.title("Actual Price Vs Predicted Price by RF Regressor")
plt.legend()
plt.show()

In [ ]:
# coefficients of random forest
rf_coefficient = pd.Series(rf_regressor.feature_importances_, index=X.columns.tolist())

rf_coefficient.sort_values(ascending=False)

## Model comparison

In [ ]:
models_df = pd.DataFrame({
    'model': "Linear KNN DecisionTree RandomForerst".split(),
    'r2_score': [lr_score, knn_score, dt_score, rf_score,],
    'mae': [lr_mae, knn_mae, dt_mae, rf_mae ]
})

models_df

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(20,5))
fig.suptitle("Regression Model Comparision")
ax[0].plot(models_df.model, models_df.r2_score, color='black', marker='^')
ax[0].set_title("R^2")
ax[1].plot(models_df.model, models_df.mae, color='r', marker='o')
ax[1].set_title("Mean Absolute Error")
plt.show()